In [1]:
import pandas as pd
import bokeh.plotting as bpl
import bokeh.models as bmd
import bokeh.layouts as bly
import bokeh.transform as btr
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np

In [2]:
data = pd.read_csv("http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip", compression = "zip", encoding = "latin1")

In [3]:
pio.templates.default = "plotly_white"
pio.templates


Templates configuration
-----------------------
    Default template: 'plotly_white'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [4]:
data 

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-12-22,071735,2,9,21,2,21,21,114,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
1,2020-12-22,1c4583,2,12,9,2,9,9,4,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
2,2020-12-22,0d55c9,2,12,9,1,9,9,16,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2020-12-22,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
4,2020-12-22,043f64,2,4,9,2,9,9,10,2,...,99,1,1,2,97,3,99,MÃ©xico,97,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420210,2020-12-22,24218c,2,12,9,1,9,9,5,1,...,2,2,97,1,2,7,99,MÃ©xico,97,97
3420211,2020-12-22,342996,2,4,20,2,20,20,184,1,...,2,1,3,2,97,6,99,MÃ©xico,97,97
3420212,2020-12-22,42ca4e,2,4,9,1,9,9,7,1,...,2,1,3,2,97,6,99,MÃ©xico,97,97
3420213,2020-12-22,40bd08,2,12,9,1,9,9,16,1,...,1,2,97,1,2,7,99,MÃ©xico,97,97


In [5]:
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [6]:
data[["FECHA_ACTUALIZACION", "FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF"]] #COLUMNA QUE ME INTERESA DataFrame de una columna -----> Serie

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,FECHA_INGRESO,FECHA_DEF
0,2020-12-22,2020-03-12,2020-03-18,9999-99-99
1,2020-12-22,2020-03-30,2020-03-30,9999-99-99
2,2020-12-22,2020-03-24,2020-03-26,9999-99-99
3,2020-12-22,2020-03-27,2020-04-02,9999-99-99
4,2020-12-22,2020-03-26,2020-03-26,2020-03-30
...,...,...,...,...
3420210,2020-12-22,2020-12-07,2020-12-15,9999-99-99
3420211,2020-12-22,2020-12-12,2020-12-15,9999-99-99
3420212,2020-12-22,2020-12-13,2020-12-15,9999-99-99
3420213,2020-12-22,2020-11-30,2020-12-02,9999-99-99


In [7]:
data[['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN']]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN
0,2020-12-22,071735,2
1,2020-12-22,1c4583,2
2,2020-12-22,0d55c9,2
3,2020-12-22,1468a5,1
4,2020-12-22,043f64,2
...,...,...,...
3420210,2020-12-22,24218c,2
3420211,2020-12-22,342996,2
3420212,2020-12-22,42ca4e,2
3420213,2020-12-22,40bd08,2


In [8]:
data.loc[10] #INFORMACION DE LA FILA 10

FECHA_ACTUALIZACION      2020-12-22
ID_REGISTRO                  0955a5
ORIGEN                            2
SECTOR                           12
ENTIDAD_UM                        6
SEXO                              1
ENTIDAD_NAC                      14
ENTIDAD_RES                       6
MUNICIPIO_RES                    10
TIPO_PACIENTE                     1
FECHA_INGRESO            2020-04-01
FECHA_SINTOMAS           2020-03-28
FECHA_DEF                9999-99-99
INTUBADO                         97
NEUMONIA                          2
EDAD                             38
NACIONALIDAD                      1
EMBARAZO                          2
HABLA_LENGUA_INDIG                2
INDIGENA                          2
DIABETES                          2
EPOC                              2
ASMA                              2
INMUSUPR                          2
HIPERTENSION                      2
OTRA_COM                          2
CARDIOVASCULAR                    2
OBESIDAD                    

In [9]:
data.loc[[1,3,6]]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
1,2020-12-22,1c4583,2,12,9,2,9,9,4,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2020-12-22,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
6,2020-12-22,13757c,1,12,15,1,15,15,106,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97


In [10]:
#Casos negativos, sospechosos y activos
data["CLASIFICACION_FINAL"].value_counts()

7    1679953
3    1287514
6     339050
5      53144
1      48840
4       9642
2       2072
Name: CLASIFICACION_FINAL, dtype: int64

In [11]:
#Casos negativos ---> 7
condicion_negativos = data["CLASIFICACION_FINAL"] == 7

In [12]:
condicion_negativos.value_counts()

False    1740262
True     1679953
Name: CLASIFICACION_FINAL, dtype: int64

In [13]:
 negativos = data[condicion_negativos] #Filtrando el data FRAME para casos negativos
 print("Casos Negativos =",len(negativos))

Casos Negativos = 1679953


In [14]:
# 1,2,3 CASOS CONFIRMADOS
confirmados = data[data["CLASIFICACION_FINAL"].isin([1,2,3])]
print("Casos confirmados =",len(confirmados))

Casos confirmados = 1338426


In [15]:
confirmados["CLASIFICACION_FINAL"].value_counts()

3    1287514
1      48840
2       2072
Name: CLASIFICACION_FINAL, dtype: int64

In [16]:
#CASOS SORPECHOSOS 6 --> sospechosos sin muestra", "sospechosos con posibilidad de resultado" y "sospechosos sin posibilidad de resultado".
sospechosos = data[data["CLASIFICACION_FINAL"] == 6]
print("Sospechosos Totales = ", len(sospechosos))

Sospechosos Totales =  339050


In [17]:
sospechosos_sin_muestra=data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==97)]
#sospechosos_sin_muestra
print("Sospechosos sin muestra =",len(sospechosos_sin_muestra))

Sospechosos sin muestra = 217141


In [18]:
sospechosos_con_posibilidad =data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==3)]
#sospechosos_sin_muestra
print("Sospechosos con posibilidad =",len(sospechosos_con_posibilidad))

Sospechosos con posibilidad = 46359


In [19]:
sospechosos_sin_posibilidad=data[(data["CLASIFICACION_FINAL"].isin([6,5,4]))&(data["RESULTADO_LAB"].isin([4]))]
print("Sospechosos sin posibilidad =", len(sospechosos_sin_posibilidad))

Sospechosos sin posibilidad = 138336


In [20]:
#Buscando defunciones 
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [21]:
#data["FECHA_DEF"].value_counts()

In [22]:
defunciones = confirmados[confirmados["FECHA_DEF"] != "9999-99-99"]
print("Defunciones =", len(defunciones))


Defunciones = 119495


In [23]:
grupos = confirmados.groupby("FECHA_INGRESO")  #agrupando por fecha
confirmados_por_fecha = grupos.size().to_frame("confirmados").reset_index()   #INICIALMENTE grupos.size() es una serie con la fecha como ind
confirmados_por_fecha

,FECHA_INGRESO,confirmados
0,2020-01-13,1
1,2020-01-28,1
2,2020-01-29,1
3,2020-02-10,1
4,2020-02-20,1
...,...,...
299,2020-12-18,7389
300,2020-12-19,3280
301,2020-12-20,993
302,2020-12-21,3852


In [24]:
confirmados_por_fecha.dtypes

FECHA_INGRESO    object
confirmados       int64
dtype: object

In [25]:
confirmados_por_fecha["FECHA_INGRESO"][:3]

0    2020-01-13
1    2020-01-28
2    2020-01-29
Name: FECHA_INGRESO, dtype: object

In [26]:
confirmados_por_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")
confirmados_por_fecha

,FECHA_INGRESO,confirmados,fecha_ingreso
0,2020-01-13,1,2020-01-13
1,2020-01-28,1,2020-01-28
2,2020-01-29,1,2020-01-29
3,2020-02-10,1,2020-02-10
4,2020-02-20,1,2020-02-20
...,...,...,...
299,2020-12-18,7389,2020-12-18
300,2020-12-19,3280,2020-12-19
301,2020-12-20,993,2020-12-20
302,2020-12-21,3852,2020-12-21


In [27]:
confirmados_por_fecha.dtypes

FECHA_INGRESO            object
confirmados               int64
fecha_ingreso    datetime64[ns]
dtype: object

In [28]:
defunciones_por_fecha = defunciones.groupby("FECHA_DEF").size().to_frame("defunciones").reset_index()
defunciones_por_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_fecha


,FECHA_DEF,defunciones,fecha_defuncion
0,2020-03-03,1,2020-03-03
1,2020-03-13,1,2020-03-13
2,2020-03-17,1,2020-03-17
3,2020-03-18,2,2020-03-18
4,2020-03-20,1,2020-03-20
...,...,...,...
276,2020-12-18,358,2020-12-18
277,2020-12-19,369,2020-12-19
278,2020-12-20,277,2020-12-20
279,2020-12-21,84,2020-12-21


In [29]:
sospechosos_por_fecha = sospechosos.groupby("FECHA_SINTOMAS").size().to_frame("sospechosos").reset_index()
sospechosos_por_fecha["fecha_sintomas"] = pd.to_datetime(sospechosos_por_fecha["FECHA_SINTOMAS"], format = "%Y-%m-%d")


In [30]:
bpl.output_notebook()

Loading BokehJS ...

In [31]:
# Ploting each line in different figures ...
def plotCases(abscisa, ordenada, x_title, y_title, g_title, paint ):
    fig_1 = px.line( x = abscisa, y = ordenada, title = g_title, width = 1500, height = 600) # labels parameter can be added with a dict

    fig_1.update_traces(marker = dict(color = paint), line = dict(color = paint, width = 2))
    fig_1.update_xaxes(showgrid = True, title = x_title)
    fig_1.update_yaxes(showgrid = True, title = y_title)
    fig_1.show()
    
plotCases(confirmados_por_fecha["fecha_ingreso"], confirmados_por_fecha["confirmados"], 
        "Fecha de registro", "Casos confirmados", "Casos de COVID-19 en México","#b4004e")
plotCases(defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones"], 
        "Fecha de defunción", "Defunciones", "Defunciones por COVID-19 en México" , "black")
# plotCases(sospechosos_por_fecha["fecha_sintomas"],sospechosos_por_fecha["sospechosos"], 
#         "Fecha del inicio de sintomas", "Casos sospechosos", "Casos sospechosos de COVID-19 en México", "#c67c00")


In [32]:
# Computing the comulative cases ...
def commulative_cases(data):
    data_vec = np.array(data)
    data_acumulados = np.cumsum(data_vec)    
    return pd.Series(data_acumulados)                           # Numpy array to pandas Serie

# Commulative confirmed cases ....
confirmados_por_fecha["casos_acumulados"] = commulative_cases(confirmados_por_fecha["confirmados"])
# Commulative Deaths  ....
defunciones_por_fecha["defunciones_acumuladas"] = commulative_cases(defunciones_por_fecha["defunciones"])

In [72]:
#Computing the average whith the actual day plus the 6 previous days
def average_of_seven_days(serie):
    data_vect = np.array(serie)
    average = np.zeros(data_vect.size)
    counter = 0
    init = 0

    for i in range(data_vect.size):
        if i >= 6:
            average[counter] = np.mean(data_vect[init:i+1])
            init += 1
        else:
            average[counter] = 1/7 * np.sum(data_vect[0:i+1])

        counter += 1

    return average

average_of_seven_days(pd.Series([1,2,3,4,5,6,7,8,9,10]))

array([0.14285714, 0.42857143, 0.85714286, 1.42857143, 2.14285714,
       3.        , 4.        , 5.        , 6.        , 7.        ])

In [33]:
# Bar plots for confirmed and death cases .....
fig_bar1_conf = px.bar(x =confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
                    labels= dict(x = "Fecha de registro", y = "Nuevos casos confirmados"), title="Nuevos casos de COVID-19 reportados diariamente", width = 900)

fig_bar1_conf.update_traces( marker = dict(color = "#ff1744"))
fig_bar1_conf.show()

fig_bar2_def = px.bar(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"],
                        labels = dict(x = "Fecha de defunción", y = "Defunciones"), title = "Nuevas defunciones por COVID-19 reportadas diariamente", width = 900)

fig_bar2_def.update_traces(marker = dict(color = "black"))
fig_bar2_def.show()

In [35]:
# Ploting the commulative confirmed and deaths cases ...

def acum_plot(x_conf_data, y_conf_data, x_def_data, y_def_data, region ):
    fig_acum = go.Figure()

    fig_acum.add_trace(go.Scatter(x = x_conf_data, y = y_conf_data,
                    mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

    fig_acum.add_trace(go.Scatter(x = x_def_data, y = y_def_data,
                    mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))

    fig_acum.update_xaxes(title = "Fecha de registro")
    fig_acum.update_yaxes(title = "Número de casos acumulados")
    fig_acum.update_layout(title = "Casos acumulados en " + region )
    fig_acum.show()


# fig_acum = go.Figure()

# fig_acum.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["casos_acumulados"],
#                     mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

# fig_acum.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones_acumuladas"],
#                     mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))



# fig_acum.update_xaxes(title = "Fecha de registro")
# fig_acum.update_yaxes(title = "Número de casos acumulados")
# fig_acum.update_layout(title = "Casos acumulados en México")
# fig_acum.show()

acum_plot(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["casos_acumulados"],
            defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones_acumuladas"], region = "México" )


In [36]:

# Ploting each line in the same figure ...

# fig_2 = px.line(confirmados_por_fecha, x = "fecha_sintomas", y = "confirmados", width = 1000, height= 300,
#                  labels= dict(fecha_sintomas = "Fecha", confirmados = "Número de personas", name = "Casos confirmados"))

fig_2 = go.Figure()

fig_2.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
                mode = "lines",name = "Casos confirmados", line = dict(color = "#b4004e")) )
fig_2.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"], 
                mode = "lines",name = "Defunciones", line = dict(color = "black") ) )
fig_2.add_trace(go.Scatter(x = sospechosos_por_fecha["fecha_sintomas"], y = sospechosos_por_fecha["sospechosos"],
                mode = "lines", name = "Casos sospechosos", line = dict(color = "#c67c00")))

fig_2.update_xaxes(showgrid = True, title = "Fecha")
fig_2.update_yaxes(showgrid = True , title = "Número de personas")
fig_2.update_layout( width = 1200, height= 400 , title = "COVID-19 México")
fig_2.show()

In [37]:
# Distribution of cases per state ...................................
confirmados

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-12-22,071735,2,9,21,2,21,21,114,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
1,2020-12-22,1c4583,2,12,9,2,9,9,4,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
2,2020-12-22,0d55c9,2,12,9,1,9,9,16,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2020-12-22,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
4,2020-12-22,043f64,2,4,9,2,9,9,10,2,...,99,1,1,2,97,3,99,MÃ©xico,97,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420196,2020-12-22,2dc50c,2,12,9,2,15,15,25,1,...,2,2,97,1,1,3,99,MÃ©xico,97,97
3420200,2020-12-22,3ad95a,2,10,9,1,9,9,16,2,...,2,2,97,2,97,1,99,MÃ©xico,97,2
3420201,2020-12-22,3e71e3,2,12,27,1,27,27,4,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97
3420203,2020-12-22,299ae5,2,12,22,1,22,22,14,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97


In [38]:
#Grouping by state ENTIDAD_RES
confirmados_por_estado = confirmados.groupby("ENTIDAD_RES").size().to_frame("Casos confirmados").reset_index()
confirmados_por_estado

,ENTIDAD_RES,Casos confirmados
0,1,16238
1,2,31807
2,3,16561
3,4,7118
4,5,47132
5,6,7685
6,7,8084
7,8,35278
8,9,297345
9,10,24301


In [39]:
# Cargando el diccionario de estados
dics = pd.read_excel("../diccionarios/201128 Catalogos_.xlsx", sheet_name = None)
dics.keys()

dict_keys(['Catálogo ORIGEN', 'Catálogo SECTOR', 'Catálogo SEXO', 'Catálogo TIPO_PACIENTE', 'Catálogo SI_NO', 'Catálogo NACIONALIDAD', 'Catálogo RESULTADO_LAB', 'Catálogo RESULTADO_ANTIGENO', 'Catálogo CLASIFICACION_FINAL', 'Catálogo de ENTIDADES', 'Catálogo MUNICIPIOS'])

In [40]:
dic_estados = dics["Catálogo de ENTIDADES"]
dic_estados

,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,AGUASCALIENTES,AS
1,2,BAJA CALIFORNIA,BC
2,3,BAJA CALIFORNIA SUR,BS
3,4,CAMPECHE,CC
4,5,COAHUILA DE ZARAGOZA,CL
5,6,COLIMA,CM
6,7,CHIAPAS,CS
7,8,CHIHUAHUA,CH
8,9,CIUDAD DE MÉXICO,DF
9,10,DURANGO,DG


In [41]:
confirmados_por_entidad = pd.merge(confirmados_por_estado, dic_estados, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD" ) # left outer join
confirmados_por_entidad

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,16238,1,AGUASCALIENTES,AS
1,2,31807,2,BAJA CALIFORNIA,BC
2,3,16561,3,BAJA CALIFORNIA SUR,BS
3,4,7118,4,CAMPECHE,CC
4,5,47132,5,COAHUILA DE ZARAGOZA,CL
5,6,7685,6,COLIMA,CM
6,7,8084,7,CHIAPAS,CS
7,8,35278,8,CHIHUAHUA,CH
8,9,297345,9,CIUDAD DE MÉXICO,DF
9,10,24301,10,DURANGO,DG


In [42]:
confirmados_por_entidad.dtypes

ENTIDAD_RES            int64
Casos confirmados      int64
CLAVE_ENTIDAD          int64
ENTIDAD_FEDERATIVA    object
ABREVIATURA           object
dtype: object

In [43]:
# Bar plot of the confirmed cases by stte
confirmados_por_entidad.sort_values("Casos confirmados", ascending = False, inplace = True ) # Ordenando por número de casos de forma descendente 
fig_bar_edo = px.bar( x = confirmados_por_entidad["ENTIDAD_FEDERATIVA"], y = confirmados_por_entidad["Casos confirmados"],
                        labels = dict(x = "Entidad Federativa", y = "Número de casos confirmados"), title = "Número de casos confirmados por estado")


fig_bar_edo.update_xaxes(type='category', tickfont=dict(size = 10))
fig_bar_edo.update_traces(marker = dict(color = "#b71c1c"), opacity = 0.8)
fig_bar_edo.show()

In [44]:
poblaciones = pd.read_csv("http://www.conapo.gob.mx/work/models/CONAPO/Datos_Abiertos/Proyecciones2018/pob_mit_proyecciones.csv", encoding="latin1")
poblaciones

,RENGLON,AÑO,ENTIDAD,CVE_GEO,EDAD,SEXO,POBLACION
0,1,1950,República Mexicana,0,0,Hombres,572103
1,2,1950,República Mexicana,0,0,Mujeres,559162
2,3,1950,República Mexicana,0,1,Hombres,514540
3,4,1950,República Mexicana,0,1,Mujeres,505269
4,5,1950,República Mexicana,0,2,Hombres,478546
...,...,...,...,...,...,...,...
592455,592456,2050,Zacatecas,32,107,Mujeres,4
592456,592457,2050,Zacatecas,32,108,Hombres,1
592457,592458,2050,Zacatecas,32,108,Mujeres,2
592458,592459,2050,Zacatecas,32,109,Hombres,0


In [45]:
poblacion_2020 = poblaciones[poblaciones["AÑO"] == 2020]

In [46]:
grupos_pob = poblacion_2020.groupby(["CVE_GEO","ENTIDAD"])
poblacion_tot_estado = grupos_pob["POBLACION"].sum().to_frame("POBLACION").reset_index()   # Sum operation in the POBLACION column of each group

In [47]:
datos_por_estado = pd.merge(confirmados_por_entidad, poblacion_tot_estado, how = "left", left_on = "CLAVE_ENTIDAD", right_on = "CVE_GEO" )
datos_por_estado.columns

Index(['ENTIDAD_RES', 'Casos confirmados', 'CLAVE_ENTIDAD',
       'ENTIDAD_FEDERATIVA', 'ABREVIATURA', 'CVE_GEO', 'ENTIDAD', 'POBLACION'],
      dtype='object')

In [48]:
# Rate of confirmed case by 100,000 people 
datos_por_estado["TASA DE CASOS POR 100K"] = 100000 * datos_por_estado["Casos confirmados"]/datos_por_estado["POBLACION"]
datos_por_estado.sort_values("TASA DE CASOS POR 100K", ascending = False, inplace = True)
datos_por_estado.reset_index(drop = True, inplace = True)
datos_por_estado

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA,CVE_GEO,ENTIDAD,POBLACION,TASA DE CASOS POR 100K
0,9,297345,9,CIUDAD DE MÉXICO,DF,9,Ciudad de México,9018645,3297.003042
1,3,16561,3,BAJA CALIFORNIA SUR,BS,3,Baja California Sur,804708,2058.013590
2,27,42112,27,TABASCO,TC,27,Tabasco,2572287,1637.142356
3,26,49063,26,SONORA,SR,26,Sonora,3074745,1595.677040
4,5,47132,5,COAHUILA DE ZARAGOZA,CL,5,Coahuila,3218720,1464.308794
5,19,78362,19,NUEVO LEÓN,NL,19,Nuevo León,5610153,1396.789000
6,24,38649,24,SAN LUIS POTOSÍ,SP,24,San Luis Potosí,2866142,1348.467731
7,10,24301,10,DURANGO,DG,10,Durango,1868996,1300.216801
8,22,29527,22,QUERÉTARO,QT,22,Querétaro,2279637,1295.250077
9,11,77557,11,GUANAJUATO,GT,11,Guanajuato,6228175,1245.260449


In [49]:
# Bar plot of the rate of confirmed cases per 100k persons 
fig_bar_cases_rate = px.bar(x = datos_por_estado["ENTIDAD"], y = datos_por_estado["TASA DE CASOS POR 100K"],
                            labels = dict(y = "Casos confirmados / 100k personas" , x = "") ,title = "Tasa de casos confirmados por cada 100,000 personas")


fig_bar_cases_rate.update_traces(marker = dict(color = "#ff616f"))
#fig_bar_cases_rate.update_xaxes(tickfont = dict(size = 11))
fig_bar_cases_rate.show()

In [50]:
# Groping by state ENTIDAD_RES and FECHA_INGRESO
confirmados_por_estado_fecha = confirmados.groupby(["ENTIDAD_RES","FECHA_INGRESO"]).size().to_frame("CONFIRMADOS").reset_index()
confirmados_por_estado_fecha["Fecha ingreso"] = pd.to_datetime(confirmados_por_estado_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")


confirmados_por_estado_fecha

,ENTIDAD_RES,FECHA_INGRESO,CONFIRMADOS,Fecha ingreso
0,1,2020-02-20,1,2020-02-20
1,1,2020-03-14,1,2020-03-14
2,1,2020-03-17,1,2020-03-17
3,1,2020-03-18,3,2020-03-18
4,1,2020-03-19,1,2020-03-19
...,...,...,...,...
8879,32,2020-12-17,188,2020-12-17
8880,32,2020-12-18,73,2020-12-18
8881,32,2020-12-19,76,2020-12-19
8882,32,2020-12-20,38,2020-12-20


In [51]:
defunciones_por_estado_fecha = defunciones.groupby(["ENTIDAD_RES", "FECHA_DEF"]).size().to_frame("DEFUNCIONES").reset_index()
defunciones_por_estado_fecha["Fecha defuncion"] = pd.to_datetime(defunciones_por_estado_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_estado_fecha

,ENTIDAD_RES,FECHA_DEF,DEFUNCIONES,Fecha defuncion
0,1,2020-04-10,1,2020-04-10
1,1,2020-04-13,1,2020-04-13
2,1,2020-04-17,1,2020-04-17
3,1,2020-04-19,1,2020-04-19
4,1,2020-04-22,1,2020-04-22
...,...,...,...,...
7838,32,2020-12-17,6,2020-12-17
7839,32,2020-12-18,9,2020-12-18
7840,32,2020-12-19,5,2020-12-19
7841,32,2020-12-20,2,2020-12-20


In [52]:
entidad_clave = datos_por_estado["ENTIDAD"].to_frame()
entidad_clave["CLAVE_ENTIDAD"] = datos_por_estado["CLAVE_ENTIDAD"]
# Merge between entidad_cave and confirmados_por_estado_fecha
confirmados_por_estado_fecha_clave = pd.merge(confirmados_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")
confirmados_por_estado_fecha_clave


,ENTIDAD_RES,FECHA_INGRESO,CONFIRMADOS,Fecha ingreso,ENTIDAD,CLAVE_ENTIDAD
0,1,2020-02-20,1,2020-02-20,Aguascalientes,1
1,1,2020-03-14,1,2020-03-14,Aguascalientes,1
2,1,2020-03-17,1,2020-03-17,Aguascalientes,1
3,1,2020-03-18,3,2020-03-18,Aguascalientes,1
4,1,2020-03-19,1,2020-03-19,Aguascalientes,1
...,...,...,...,...,...,...
8879,32,2020-12-17,188,2020-12-17,Zacatecas,32
8880,32,2020-12-18,73,2020-12-18,Zacatecas,32
8881,32,2020-12-19,76,2020-12-19,Zacatecas,32
8882,32,2020-12-20,38,2020-12-20,Zacatecas,32


In [53]:
# Merge betweem entidad_clave and defunciones_por_estado_fecha
defunciones_por_estado_fecha_clave = pd.merge(defunciones_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")
defunciones_por_estado_fecha_clave

,ENTIDAD_RES,FECHA_DEF,DEFUNCIONES,Fecha defuncion,ENTIDAD,CLAVE_ENTIDAD
0,1,2020-04-10,1,2020-04-10,Aguascalientes,1
1,1,2020-04-13,1,2020-04-13,Aguascalientes,1
2,1,2020-04-17,1,2020-04-17,Aguascalientes,1
3,1,2020-04-19,1,2020-04-19,Aguascalientes,1
4,1,2020-04-22,1,2020-04-22,Aguascalientes,1
...,...,...,...,...,...,...
7838,32,2020-12-17,6,2020-12-17,Zacatecas,32
7839,32,2020-12-18,9,2020-12-18,Zacatecas,32
7840,32,2020-12-19,5,2020-12-19,Zacatecas,32
7841,32,2020-12-20,2,2020-12-20,Zacatecas,32


In [54]:
confirmados_mich = confirmados_por_estado_fecha_clave[confirmados_por_estado_fecha_clave["ENTIDAD_RES"] == 16]
print("Total de casos confirmados = ",confirmados_mich["CONFIRMADOS"].sum())
confirmados_mich.reset_index(drop = True, inplace = True)
confirmados_mich["CONFIRMADOS_ACUMULADOS"] = commulative_cases(confirmados_mich["CONFIRMADOS"]) # Accumulated confirmed cases in Mich.
confirmados_mich

Total de casos confirmados =  31779


,ENTIDAD_RES,FECHA_INGRESO,CONFIRMADOS,Fecha ingreso,ENTIDAD,CLAVE_ENTIDAD,CONFIRMADOS_ACUMULADOS
0,16,2020-03-18,1,2020-03-18,Michoacán,16,1
1,16,2020-03-19,1,2020-03-19,Michoacán,16,2
2,16,2020-03-20,2,2020-03-20,Michoacán,16,4
3,16,2020-03-21,5,2020-03-21,Michoacán,16,9
4,16,2020-03-23,3,2020-03-23,Michoacán,16,12
...,...,...,...,...,...,...,...
272,16,2020-12-17,160,2020-12-17,Michoacán,16,31670
273,16,2020-12-18,84,2020-12-18,Michoacán,16,31754
274,16,2020-12-19,21,2020-12-19,Michoacán,16,31775
275,16,2020-12-20,1,2020-12-20,Michoacán,16,31776


In [55]:
defunciones_mich = defunciones_por_estado_fecha_clave[defunciones_por_estado_fecha_clave["ENTIDAD_RES"] == 16]
print("Total de defunciones acumuladas = ", defunciones_mich["DEFUNCIONES"].sum())
defunciones_mich.reset_index(drop = True, inplace = True)     
defunciones_mich["DEFUNCIONES_ACUMULADAS"] = commulative_cases(defunciones_mich["DEFUNCIONES"])
defunciones_mich

Total de defunciones acumuladas =  2616


,ENTIDAD_RES,FECHA_DEF,DEFUNCIONES,Fecha defuncion,ENTIDAD,CLAVE_ENTIDAD,DEFUNCIONES_ACUMULADAS
0,16,2020-03-26,1,2020-03-26,Michoacán,16,1
1,16,2020-04-03,2,2020-04-03,Michoacán,16,3
2,16,2020-04-04,1,2020-04-04,Michoacán,16,4
3,16,2020-04-06,1,2020-04-06,Michoacán,16,5
4,16,2020-04-08,1,2020-04-08,Michoacán,16,6
...,...,...,...,...,...,...,...
252,16,2020-12-16,7,2020-12-16,Michoacán,16,2591
253,16,2020-12-17,11,2020-12-17,Michoacán,16,2602
254,16,2020-12-18,7,2020-12-18,Michoacán,16,2609
255,16,2020-12-19,2,2020-12-19,Michoacán,16,2611


In [56]:
#
acum_plot(confirmados_mich["Fecha ingreso"],confirmados_mich["CONFIRMADOS_ACUMULADOS"],
            defunciones_mich["Fecha defuncion"], defunciones_mich["DEFUNCIONES_ACUMULADAS"], region = confirmados_mich["ENTIDAD"].iloc[0] )
print("Total de defunciones acumuladas = ", defunciones_mich["DEFUNCIONES"].sum())
print("Total de casos confirmados = ",confirmados_mich["CONFIRMADOS"].sum())

Total de defunciones acumuladas =  2616
Total de casos confirmados =  31779
